# 20_ 카카오 OpenAPI를 연동한 이미지 수집(1)

## #01. 기본 준비 단계

### 1) 필요한 패키지 가져오기 

In [1]:
import requests
import json
import urllib
import os
import datetime as dt
import pandas as pd
from pandas import DataFrame

### 2) Open API 연동키 정의 

In [2]:
api_key = "681e20ed293a6f6822156c7de5162153"

### 3) 검색어 및 페이지 구현에 필요한 변수 준비 

In [3]:
q = "박보영"
page = 5 
size = 80

### 4) 이미지가 저장될 폴더의 이름 만들기 

#### 검색어 + 연재 날짜 형식의 폴더 이름 준비

In [4]:
datetime = dt.datetime.now().strftime("%y%m%d_%H%M%S")
dirname = "%s_%s"  %(q, datetime)
dirname

'박보영_200328_104013'

#### 폴더 생성

In [6]:
if not os.path.exists(dirname):
    os.mkdir(dirname)

## #02.Open API 연동

### 1) 접속 세션 만들기 

In [7]:
# 접속 세션 만들기 
user_agent = "Mozilla/5.0 (Linux; Android 5.0; SM-G900P Build/LRX21T) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Mobile Safari/537.36"

session = requests.Session()

### 2) 세션의 HTTP 헤더 설정
인증키를 header에 포함 시켜야 함  "KaKaoAK" 뒤에 공백 추가 주의 

In [8]:
session.headers.update({'User-agent':user_agent, 'referer':None, 'Authorization':'KakaoAK '+api_key})

### 3) API에 전달할 파라미터 인코딩

인터넷 주소에는 한글이나 공백이 포함 될 수 없기 때문에 검색어에 대해 인코딩 처리를 수행해야 한다.

In [9]:
params = {"page":page, "size":size,"query":q}
query = urllib.parse.urlencode(params)
query

'page=5&size=80&query=%EB%B0%95%EB%B3%B4%EC%98%81'

### 4) 최종 접속 주소 구성

In [10]:
url_tpl = "https://dapi.kakao.com/v2/search/image"
api_url = url_tpl + "?" + query
api_url

'https://dapi.kakao.com/v2/search/image?page=5&size=80&query=%EB%B0%95%EB%B3%B4%EC%98%81'

## #03. API 검색 결과 수집

### 1) API에 접근하여 데이터 가져오기 

In [11]:
r = session.get(api_url)

if r.status_code != 200 :
    print("[%d Error] %s" % (r.status_code, r.reason))
    quit()

### 2) 가져온 결과를 딕셔너리로 변환

In [12]:
r.encoding = "utf-8"
image_dict = json.loads(r.text)
image_dict

{'documents': [{'collection': 'blog',
   'datetime': '2011-08-19T12:44:00.000+09:00',
   'display_sitename': '네이버블로그',
   'doc_url': 'http://blog.naver.com/newsone7/20135678000',
   'height': 434,
   'image_url': 'http://postfiles2.naver.net/20110819_113/newsone7_1313725434260czqmS_JPEG/%B9%DA%BA%B8%BF%B53.jpg?type=w3',
   'thumbnail_url': 'https://search4.kakaocdn.net/argon/130x130_85_c/BYJxxY54nl7',
   'width': 353},
  {'collection': 'blog',
   'datetime': '2014-09-26T22:44:00.000+09:00',
   'display_sitename': '네이버블로그',
   'doc_url': 'http://blog.naver.com/thankyoumom2/220133842876',
   'height': 578,
   'image_url': 'http://postfiles13.naver.net/20140926_28/puppydog05_1411738928800M6O5H_JPEG/%B9%DA%BA%B8%BF%B55.jpg?type=w2',
   'thumbnail_url': 'https://search4.kakaocdn.net/argon/130x130_85_c/IrMfAq500bV',
   'width': 725},
  {'collection': 'blog',
   'datetime': '2009-04-22T12:28:00.000+09:00',
   'display_sitename': '네이버블로그',
   'doc_url': 'http://blog.naver.com/jtl2553/140066903

### 3) DataFrame 생성

#### 딕셔너리 중에서 검색 결과에 해당하는 documents에 대한 부분을 DataFrame 으로 변환한다.

In [13]:
image_df = DataFrame(image_dict['documents'])
image_df

,collection,datetime,display_sitename,doc_url,height,image_url,thumbnail_url,width
0,blog,2011-08-19T12:44:00.000+09:00,네이버블로그,http://blog.naver.com/newsone7/20135678000,434,http://postfiles2.naver.net/20110819_113/newso...,https://search4.kakaocdn.net/argon/130x130_85_...,353
1,blog,2014-09-26T22:44:00.000+09:00,네이버블로그,http://blog.naver.com/thankyoumom2/220133842876,578,http://postfiles13.naver.net/20140926_28/puppy...,https://search4.kakaocdn.net/argon/130x130_85_...,725
2,blog,2009-04-22T12:28:00.000+09:00,네이버블로그,http://blog.naver.com/jtl2553/140066903745,701,http://postfiles1.naver.net/data42/2009/4/22/2...,https://search3.kakaocdn.net/argon/130x130_85_...,545
3,blog,2012-07-17T14:28:00.000+09:00,네이버블로그,http://blog.naver.com/002humanist/70142510620,503,http://postfiles13.naver.net/20120717_156/002h...,https://search3.kakaocdn.net/argon/130x130_85_...,362
4,blog,2012-11-02T18:12:00.000+09:00,네이버블로그,http://blog.naver.com/qudgml331/120172409498,304,http://postfiles4.naver.net/20121102_67/qudgml...,https://search4.kakaocdn.net/argon/130x130_85_...,540
...,...,...,...,...,...,...,...,...
75,news,2019-04-29T11:15:21.000+09:00,스타뉴스,http://v.media.daum.net/v/20190429111521412,1536,https://t1.daumcdn.net/news/201904/29/starnews...,https://search2.kakaocdn.net/argon/130x130_85_...,1024
76,news,2018-08-07T17:06:02.000+09:00,스타뉴스,http://v.media.daum.net/v/20180807170602975,365,https://t1.daumcdn.net/news/201808/07/starnews...,https://search1.kakaocdn.net/argon/130x130_85_...,560
77,blog,2015-06-29T17:44:00.000+09:00,네이버블로그,http://blog.naver.com/cheoly01/220405071516,740,http://postfiles6.naver.net/20150629_293/cheol...,https://search1.kakaocdn.net/argon/130x130_85_...,550
78,blog,2015-08-11T20:28:00.000+09:00,네이버블로그,http://blog.naver.com/soomihan21/220447896267,630,http://postfiles15.naver.net/20150811_206/soom...,https://search2.kakaocdn.net/argon/130x130_85_...,416


## #04. 이미지 다운받기 

In [20]:
# 저장되는 이미지 파일의 수를 카운트 하기 위한 변수
count = 0 

# 이미지 주소에 대해서만 반복
for image_url in image_df['image_url']:
    # 카운터 증가 
    count += 1
    
    # 파일이 저장될 경로 생성
    path = "%s/%04d.jpg" %(dirname, count)
    print("[%s] >> %s" % (path, image_url))
    
    try:
        # 이미지 주소를 다운로드를 위해  stream 모드로 가져온다.
        r = session.get(image_url, stream=True)
        
        # 에러 발생시 저장이 불가능하므로 건너뛰고 반복의 조건식으로 이동
        if r.status_code != 200 :
            print("######> 저장실패1")
            continue
            
        ## 추출한 데이터를 저장
        # -> 'w' : 텍스트 쓰기 모드, 'wb':바이너리(이진값) 쓰기 모드
        with open(path, 'wb') as f:
            f.write(r.raw.read())
            print("------> 저장 성공")
    except Exception as ex:
        print("######> 저장실패2")
        print(ex)  # 에러 메세지를 강제 출력 --> 에러 원인을 확인 하기 위함.
        continue

[박보영_200328_104013/0001.jpg] >> http://postfiles2.naver.net/20110819_113/newsone7_1313725434260czqmS_JPEG/%B9%DA%BA%B8%BF%B53.jpg?type=w3
------> 저장 성공
[박보영_200328_104013/0002.jpg] >> http://postfiles13.naver.net/20140926_28/puppydog05_1411738928800M6O5H_JPEG/%B9%DA%BA%B8%BF%B55.jpg?type=w2
------> 저장 성공
[박보영_200328_104013/0003.jpg] >> http://postfiles1.naver.net/data42/2009/4/22/272/%B9%DA%BA%B8%BF%B52_jtl2553.jpg?type=w3
------> 저장 성공
[박보영_200328_104013/0004.jpg] >> http://postfiles13.naver.net/20120717_156/002humanist_1342502344602myJRr_JPEG/%B9%DA%BA%B8%BF%B508.jpg?type=w3
------> 저장 성공
[박보영_200328_104013/0005.jpg] >> http://postfiles4.naver.net/20121102_67/qudgml331_13518474382001HXiD_JPEG/%B9%DA%BA%B8%BF%B5.jpg?type=w2
------> 저장 성공
[박보영_200328_104013/0006.jpg] >> http://postfiles15.naver.net/20130513_142/oper45454_1368452621648YNxW8_JPEG/%B9%DA%BA%B8%BF%B5_%B4%C1%B4%EB%BC%D2%B3%E2_%BD%C3%BB%E7%C8%B8_121029-1200-2.jpg?type=w3
------> 저장 성공
[박보영_200328_104013/0007.jpg] >> http://t